In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import torch
import pandas as pd
from functools import partial
from src.models.utils import train_classifier, evaluate_classifier
from src.models.fish import Fish
from src.data.load import load_data
from src.data.utils import collate_batch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, Dataset
from torchtext.data.utils import get_tokenizer

### 1. Data

In [ ]:
# Hyperparameters
embed_dim = 256
batch_size = 64
EPOCHS = 10

In [2]:
train_dataset, val_dataset, test_dataset = load_data("../data/processed/")
tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
vocab = torch.load("../data/vocab.pt")

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=partial(collate_batch, vocab=vocab, tokenizer=tokenizer),
    sampler=train_dataset.get_sampler(),
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=partial(collate_batch, vocab=vocab, tokenizer=tokenizer),
    sampler=val_dataset.get_sampler(),
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=partial(collate_batch, vocab=vocab, tokenizer=tokenizer),
)

### 2. Training

In [4]:
model = Fish(len(vocab),embed_dim, 2)
train_classifier(model, train_dataloader, val_dataloader, epochs=EPOCHS)

| epoch   1 |  1000/ 2795 batches | accuracy    0.805
| epoch   1 |  2000/ 2795 batches | accuracy    0.860
-----------------------------------------------------------
| end of epoch   1 | time:  6.54s | valid accuracy    0.855 
-----------------------------------------------------------


### 3. Evaluation

In [5]:
print('Checking the results of test dataset.')
accu_test = evaluate_classifier(model, test_dataloader, CrossEntropyLoss())
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.797
